In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import datetime
import requests

In [2]:
# url = 'https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods'
# neighbourhood_list = []

# driver = webdriver.Chrome()
# driver.get(url)

In [3]:
# tables = driver.find_elements_by_class_name('wikitable')
# driver.find_elements_by_tag_name


# for table in tables:
#     names = table.find_elements_by_tag_name('td')
#     for name in names:
#         text = name.text
#         neighbourhood_list.append(text)

# driver.close()

In [4]:
# len(neighbourhood_list)

In [5]:
# df = pd.DataFrame({'Neighbourhood': neighbourhood_list[::2], 'Limits': neighbourhood_list[1::2]})

In [73]:
from pandas.io.json import json_normalize
import json

In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
data = newyork_data['features']
df = json_normalize(data)

In [8]:
df.head()

,geometry.coordinates,geometry.type,geometry_name,id,properties.annoangle,properties.annoline1,properties.annoline2,properties.annoline3,properties.bbox,properties.borough,properties.name,properties.stacked,type
0,"[-73.84720052054902, 40.89470517661]",Point,geom,nyu_2451_34572.1,0.0,Wakefield,None,None,"[-73.84720052054902, 40.89470517661, -73.84720...",Bronx,Wakefield,1,Feature
1,"[-73.82993910812398, 40.87429419303012]",Point,geom,nyu_2451_34572.2,0.0,Co-op,City,None,"[-73.82993910812398, 40.87429419303012, -73.82...",Bronx,Co-op City,2,Feature
2,"[-73.82780644716412, 40.887555677350775]",Point,geom,nyu_2451_34572.3,0.0,Eastchester,None,None,"[-73.82780644716412, 40.887555677350775, -73.8...",Bronx,Eastchester,1,Feature
3,"[-73.90564259591682, 40.89543742690383]",Point,geom,nyu_2451_34572.4,0.0,Fieldston,None,None,"[-73.90564259591682, 40.89543742690383, -73.90...",Bronx,Fieldston,1,Feature
4,"[-73.9125854610857, 40.890834493891305]",Point,geom,nyu_2451_34572.5,0.0,Riverdale,None,None,"[-73.9125854610857, 40.890834493891305, -73.91...",Bronx,Riverdale,1,Feature


In [9]:
borough = []
neighbourhood = []
neighbourhood_lat = []
neighbourhood_lon = []

for data_piece in data:
    borough.append(data_piece['properties']['borough'])
    neighbourhood.append(data_piece['properties']['name'])
    neighbourhood_lat.append(data_piece['geometry']['coordinates'][1])
    neighbourhood_lon.append(data_piece['geometry']['coordinates'][0])

new_df = pd.DataFrame({'Borough': borough, 
                       'Neighbourhood': neighbourhood,
                       'Latitude': neighbourhood_lat, 
                       'Longitude': neighbourhood_lon})

In [10]:
new_df.head()

,Borough,Latitude,Longitude,Neighbourhood
0,Bronx,40.894705,-73.847201,Wakefield
1,Bronx,40.874294,-73.829939,Co-op City
2,Bronx,40.887556,-73.827806,Eastchester
3,Bronx,40.895437,-73.905643,Fieldston
4,Bronx,40.890834,-73.912585,Riverdale


In [11]:
man_df = new_df[new_df['Borough']=='Manhattan']

In [12]:
man_df.to_csv('Manhattan Neighbourhood.csv', index=False)

In [2]:
# The code was removed by Watson Studio for sharing.

In [46]:
man_df.shape

(40, 4)

In [86]:
# Request regular data from Foursquare API

# column_names = ['Neighbourhood', 'Name', 'lat', 'lon', 'Address', 'Category', 'ID']
# venues = pd.DataFrame(columns=column_names)

# section = 'food'
# radius = 500
# LIMIT = 100

# for i in range(40):
#     latitude = man_df.iloc[i, 1]
#     longitude = man_df.iloc[i, 2]
    
#     url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&section={}&limit={}'.\
#     format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, section, LIMIT)
    
#     results = requests.get(url).json()
    
#     for result in results['response']['groups'][0]['items']:
#         info = result['venue']
#         name = info['name']
#         lat = info['location']['lat']
#         lon = info['location']['lng']
#         add = info['location']['formattedAddress']
#         venue_id = info['id']    
#         cat_1 = info['categories'][0]['shortName']

#         venues = venues.append({'Neighbourhood': man_df.iloc[i, 3],
#                                 'Name': name,
#                                 'lat': lat,
#                                 'lon': lon,
#                                 'Address': add,
#                                 'Category': cat_1,
#                                 'ID': venue_id}, ignore_index=True)

In [193]:
# Drop duplicated restaurants when that one is found near multiple neighbourhoods. Keep the first occurence.

venues.drop_duplicates(['ID'], keep='first', inplace=True)

In [194]:
venues.to_csv('Manhattan Venue.csv', index=False)

In [89]:
venues.head()

,Neighbourhood,Name,lat,lon,Address,Category,ID
0,Marble Hill,Arturo's,40.874412,-73.910271,"[5198 Broadway (at 225th St.), New York, NY 10...",Pizza,4b4429abf964a52037f225e3
1,Marble Hill,Tibbett Diner,40.880404,-73.908937,"[3033 Tibbett Ave (btwn 230th & 231st), Bronx,...",Diner,4b79cc46f964a520c5122fe3
2,Marble Hill,Dunkin' Donuts,40.877136,-73.906666,"[5501 Broadway (W 230th St), Bronx, NY 10463, ...",Donuts,4b5357adf964a520319827e3
3,Marble Hill,Land & Sea Restaurant,40.877885,-73.905873,"[5535 Broadway (231st St), Bronx, NY 10463, Un...",Seafood,4b9c9c6af964a520b27236e3
4,Marble Hill,Parrilla Latina,40.877473,-73.906073,"[230th St & Broadway, Bronx, NY 10463, United ...",Steakhouse,4dfe40df8877333e195b68fc


In [195]:
venues.shape

(2645, 7)

In [3]:
def check_key_value(key, scrape):
    if key in scrape.keys():
        a = scrape[key]
    else:
        a = np.nan
        
    return a

In [4]:
venues = pd.read_csv('Manhattan Venue.csv', header=0)

In [5]:
venues.head()

,Neighbourhood,Name,lat,lon,Address,Category,ID
0,Marble Hill,Arturo's,40.874412,-73.910271,"['5198 Broadway (at 225th St.)', 'New York, NY...",Pizza,4b4429abf964a52037f225e3
1,Marble Hill,Tibbett Diner,40.880404,-73.908937,"['3033 Tibbett Ave (btwn 230th & 231st)', 'Bro...",Diner,4b79cc46f964a520c5122fe3
2,Marble Hill,Dunkin' Donuts,40.877136,-73.906666,"['5501 Broadway (W 230th St)', 'Bronx, NY 1046...",Donuts,4b5357adf964a520319827e3
3,Marble Hill,Land & Sea Restaurant,40.877885,-73.905873,"['5535 Broadway (231st St)', 'Bronx, NY 10463'...",Seafood,4b9c9c6af964a520b27236e3
4,Marble Hill,Parrilla Latina,40.877473,-73.906073,"['230th St & Broadway', 'Bronx, NY 10463', 'Un...",Steakhouse,4dfe40df8877333e195b68fc


In [7]:
venues.Category.value_counts()[:13]

Italian          227
Pizza            155
American         128
Deli / Bodega    121
Café             119
Mexican          107
Sandwiches       105
Bakery            99
Chinese           98
French            78
Sushi             76
Japanese          72
Burgers           58
Name: Category, dtype: int64

In [9]:
venues.Category.value_counts()[11:12]

Japanese    72
Name: Category, dtype: int64

In [49]:
batch_1 = venues.Category.value_counts()[:2]

In [52]:
batch_1_index = [cat in batch_1.keys() for cat in venues.Category]

In [53]:
venues_batch_1 = venues[batch_1_index]

In [55]:
venues_batch_1.Category.value_counts()

Italian    250
Pizza      164
Name: Category, dtype: int64

In [67]:
column_names = ['ID', 'Verified', 'Category 1', 'Category 2', 'Stats', 'Price', 'Rating', 'createdAt']
venue_details = pd.DataFrame(columns=column_names)

for v_id in venues_batch_1.ID:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.\
    format(v_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    
    result = requests.get(url).json()
    info = result['response']['venue']
    
    IDs = v_id
    vrfy = info['verified']
    cat_1 = info['categories'][0]['shortName']
    
    try:
        cat_2 = info['categories'][1]['shortName']
    except IndexError:
        cat_2 = np.nan
        
    stat = check_key_value('stats', info)
    price = check_key_value('price', info)
    rates = check_key_value('rating', info)
    creation = check_key_value('createdAt', info)
    
    print(IDs, vrfy, cat_1, cat_2, stat, price, rates, creation)
    
    venue_details = venue_details.append({'ID': IDs,
                                          'Verified': vrfy,
                                          'Category 1': cat_1, 
                                          'Category 2': cat_2,
                                          'Stats': stat,
                                          'Price': price,
                                          'Rating': rates,
                                          'createdAt': creation}, ignore_index=True)   

4b4429abf964a52037f225e3 False Pizza Italian {'tipCount': 16} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 8.4 1262758315
56c3c626cd106998d2b196d0 False Pizza nan {'tipCount': 38} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 9.0 1455670822
472a027af964a520ea4b1fe3 False Italian Wine Bar {'tipCount': 74} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 8.5 1193935482
52b5d6b6498e149068fbea80 True Pizza nan {'tipCount': 45} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 8.4 1387648694
4d21107c6e8c37042b58ff9f True Italian Sicilian {'tipCount': 61} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 8.6 1294012540
5503762d498e150bef780584 False Pizza nan {'tipCount': 2} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 7.9 1426290221
4abfb6a1f964a520c29120e3 False Pizza nan {'tipCount': 11} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 6.6 1254078113
4b357135f964a520302d25e3 True Pizza nan {'tipCount': 3} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 6.0 1261793589
4cfd6

In [11]:
# batch 2
batch_2 = venues.Category.value_counts()[2:6]
batch_2_index = [cat in batch_2.keys() for cat in venues.Category]
venues_batch_2 = venues[batch_2_index]
venues_batch_2.Category.value_counts()

American         128
Deli / Bodega    121
Café             119
Mexican          107
Name: Category, dtype: int64

In [13]:
column_names = ['ID', 'Verified', 'Category 1', 'Category 2', 'Stats', 'Price', 'Rating', 'createdAt']
venue_details = pd.DataFrame(columns=column_names)

for v_id in venues_batch_2.ID:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.\
    format(v_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    
    result = requests.get(url).json()
    info = result['response']['venue']
    
    IDs = v_id
    vrfy = info['verified']
    cat_1 = info['categories'][0]['shortName']
    
    try:
        cat_2 = info['categories'][1]['shortName']
    except IndexError:
        cat_2 = np.nan
        
    stat = check_key_value('stats', info)
    price = check_key_value('price', info)
    rates = check_key_value('rating', info)
    creation = check_key_value('createdAt', info)
    
    print(IDs, vrfy, cat_1, cat_2, stat, price, rates, creation)
    
    venue_details = venue_details.append({'ID': IDs,
                                          'Verified': vrfy,
                                          'Category 1': cat_1, 
                                          'Category 2': cat_2,
                                          'Stats': stat,
                                          'Price': price,
                                          'Rating': rates,
                                          'createdAt': creation}, ignore_index=True)  

585c205665e7c70a2f1055ea False American Comfort Food {'tipCount': 1} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 7.0 1482432598
4ed7956b8b81b2bf28adc714 False Deli / Bodega Smoke Shop {'tipCount': 0} {'tier': 1, 'message': 'Cheap', 'currency': '$'} nan 1322751339
4cdd78146ad1a09302a3f756 False Deli / Bodega nan {'tipCount': 0} {'tier': 1, 'message': 'Cheap', 'currency': '$'} nan 1289582612
572bcbaa498e16431ad34a61 False American Cocktail {'tipCount': 11} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 8.5 1462487978
587996ef0923920c21ea4603 False American Café {'tipCount': 9} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 8.9 1484363503
583e5099b21014593fb544b0 False American nan {'tipCount': 21} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 8.8 1480478873
58f55261419a9e48f4dd0706 True Mexican Vegetarian / Vegan {'tipCount': 38} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 8.9 1492472417
4dfaacb9d4c01451df566572 True Mexican Cocktail {'tipCount': 157} {'tier'

In [9]:
# batch 3
batch_3 = venues.Category.value_counts()[6:11]
batch_3_index = [cat in batch_3.keys() for cat in venues.Category]
venues_batch_3 = venues[batch_3_index]
venues_batch_3.Category.value_counts()

Sandwiches    105
Bakery         99
Chinese        98
French         78
Sushi          76
Name: Category, dtype: int64

In [25]:
column_names = ['ID', 'Verified', 'Category 1', 'Category 2', 'Stats', 'Price', 'Rating', 'createdAt']
venue_details = pd.DataFrame(columns=column_names)

for v_id in venues_batch_3.ID[330:]:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.\
    format(v_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    
    result = requests.get(url).json()
    info = result['response']['venue']
    
    IDs = v_id
    vrfy = info['verified']
    cat_1 = info['categories'][0]['shortName']
    
    try:
        cat_2 = info['categories'][1]['shortName']
    except IndexError:
        cat_2 = np.nan
        
    stat = check_key_value('stats', info)
    price = check_key_value('price', info)
    rates = check_key_value('rating', info)
    creation = check_key_value('createdAt', info)
    
    print(IDs, vrfy, cat_1, cat_2, stat, price, rates, creation)
    
    venue_details = venue_details.append({'ID': IDs,
                                          'Verified': vrfy,
                                          'Category 1': cat_1, 
                                          'Category 2': cat_2,
                                          'Stats': stat,
                                          'Price': price,
                                          'Rating': rates,
                                          'createdAt': creation}, ignore_index=True)  

4a4bf8ccf964a520eeac1fe3 False Bakery nan {'tipCount': 27} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 7.7 1246492876
4a60ad34f964a52001c11fe3 False Sandwiches Italian {'tipCount': 36} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 7.2 1247849780
4c3fb5ced691c9b6f9368a0a False Sandwiches Fast Food {'tipCount': 3} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 6.5 1279243726
4b6b06dbf964a52089ed2be3 False Chinese nan {'tipCount': 3} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 6.4 1265305307
536937d4498e999c655b37c7 True Sandwiches nan {'tipCount': 27} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 8.4 1399404500
538df67d498e8697475982ee False Sandwiches Coffee Shop {'tipCount': 15} {'tier': 1, 'message': 'Cheap', 'currency': '$'} 7.8 1401812605
53d3f607498edfb90a15baf5 False Sushi nan {'tipCount': 32} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 7.6 1406400007
57bb34ed498e80a6e7fb5c30 False Bakery Café {'tipCount': 6} {'tier': 1, 'message': 'Cheap', 'currency':

In [10]:
# batch 4
batch_4 = venues.Category.value_counts()[11:12]
batch_4_index = [cat in batch_4.keys() for cat in venues.Category]
venues_batch_4 = venues[batch_4_index]
venues_batch_4.Category.value_counts()

Japanese    72
Name: Category, dtype: int64

In [11]:
column_names = ['ID', 'Verified', 'Category 1', 'Category 2', 'Stats', 'Price', 'Rating', 'createdAt']
venue_details = pd.DataFrame(columns=column_names)

for v_id in venues_batch_4.ID:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.\
    format(v_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    
    result = requests.get(url).json()
    info = result['response']['venue']
    
    IDs = v_id
    vrfy = info['verified']
    cat_1 = info['categories'][0]['shortName']
    
    try:
        cat_2 = info['categories'][1]['shortName']
    except IndexError:
        cat_2 = np.nan
        
    stat = check_key_value('stats', info)
    price = check_key_value('price', info)
    rates = check_key_value('rating', info)
    creation = check_key_value('createdAt', info)
    
    print(IDs, vrfy, cat_1, cat_2, stat, price, rates, creation)
    
    venue_details = venue_details.append({'ID': IDs,
                                          'Verified': vrfy,
                                          'Category 1': cat_1, 
                                          'Category 2': cat_2,
                                          'Stats': stat,
                                          'Price': price,
                                          'Rating': rates,
                                          'createdAt': creation}, ignore_index=True)  

57a2834b498e5060ad7eb913 True Japanese nan {'tipCount': 10} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 7.5 1470268235
531a0207498e06b4656eb8c5 False Japanese nan {'tipCount': 15} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 7.8 1394213383
542efa85498ea11026c9746c False Japanese nan {'tipCount': 15} {'tier': 4, 'message': 'Very Expensive', 'currency': '$'} 8.6 1412364933
4bd0fe22a8b3a593a123655f False Japanese nan {'tipCount': 29} {'tier': 3, 'message': 'Expensive', 'currency': '$'} 8.6 1271987746
4b3fdfc8f964a5200ab125e3 True Japanese Sushi {'tipCount': 49} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 8.5 1262477256
4bc11982b492d13ad88fa560 False Japanese nan {'tipCount': 11} {'tier': 4, 'message': 'Very Expensive', 'currency': '$'} 7.6 1270946178
4d44b985befe236acd24f8e2 False Japanese nan {'tipCount': 10} {'tier': 2, 'message': 'Moderate', 'currency': '$'} 6.5 1296349573
5aa1d85d5f68b931b6f5acda False Japanese Salad {'tipCount': 4} {'tier': 2, 'message': 'Moder

In [202]:
# Raw Data Output
# Use Batch 1, 2, 5. Somehow API output for 3 and 4 are disconnected.
venue_details.to_csv('Venue Batch 1.csv', index=False)

In [15]:
venue_details.to_csv('Venue Batch 2.csv', index=False)

In [14]:
venue_details.to_csv('Venue Batch 3.csv', index=False)

In [28]:
venue_details.to_csv('Venue Batch 4.csv', index=False)

In [35]:
venue_details = pd.concat([venue_details_11, venue_details], ignore_index=True)

In [125]:
venue_details.to_csv('Venue Batch 5.csv', index=False)

In [12]:
venue_details.to_csv('Venue Batch 6.csv', index=False)

In [136]:
# Blank

In [14]:
# Data Manipulation

# Extract price information
venue_details['Price Tier'] = [x['tier'] for x in venue_details['Price']]
venue_details['Price Description'] = [x['message'] for x in venue_details['Price']]

In [15]:
# Extract number of tips
venue_details['Tips'] = [x['tipCount'] for x in venue_details['Stats']]

In [16]:
# Extract open date and days since opening
venue_details['Open Date'] = [datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d') for x in venue_details['createdAt']]

# Measures the number of days from opening to 2019/1/1, assuming no restaurants are opened in 2019.
venue_details['Days since Open'] = \
[(datetime.datetime(2019, 1, 1) - datetime.datetime.fromtimestamp(x)).days for x in venue_details['createdAt']]

In [18]:
venue_japan = pd.merge(venues, venue_details, on='ID', how='inner')

In [19]:
venue_japan.head()

,Neighbourhood,Name,lat,lon,Address,Category,ID,Verified,Category 1,Category 2,Stats,Price,Rating,createdAt,Price Tier,Price Description,Tips,Open Date,Days since Open
0,Chinatown,Gohan,40.715143,-73.991680,"['14A Orchard St', 'New York, NY 10002', 'Unit...",Japanese,57a2834b498e5060ad7eb913,True,Japanese,NaN,{'tipCount': 10},"{'tier': 2, 'message': 'Moderate', 'currency':...",7.5,1470268235,2,Moderate,10,2016-08-04,879
1,Hamilton Heights,Geisha Japanese Cuisine,40.824355,-73.951993,"['3468 Broadway (btw. W 141 and W 142)', 'New ...",Japanese,531a0207498e06b4656eb8c5,False,Japanese,NaN,{'tipCount': 15},"{'tier': 2, 'message': 'Moderate', 'currency':...",7.8,1394213383,2,Moderate,15,2014-03-08,1759
2,Upper East Side,Kappo Masa,40.774527,-73.963466,"['976 Madison Ave (76th Street)', 'New York, N...",Japanese,542efa85498ea11026c9746c,False,Japanese,NaN,{'tipCount': 15},"{'tier': 4, 'message': 'Very Expensive', 'curr...",8.6,1412364933,4,Very Expensive,15,2014-10-04,1549
3,Yorkville,Donguri Restaurant,40.775678,-73.952478,"['309 E 83rd St Frnt 2 (btwn 1st & 2nd Ave)', ...",Japanese,4bd0fe22a8b3a593a123655f,False,Japanese,NaN,{'tipCount': 29},"{'tier': 3, 'message': 'Expensive', 'currency'...",8.6,1271987746,3,Expensive,29,2010-04-23,3174
4,Yorkville,Ichiro,40.778901,-73.950843,"['1694 2nd Ave (88th Street)', 'New York, NY 1...",Japanese,4b3fdfc8f964a5200ab125e3,True,Japanese,Sushi,{'tipCount': 49},"{'tier': 2, 'message': 'Moderate', 'currency':...",8.5,1262477256,2,Moderate,49,2010-01-03,3284


In [20]:
venue_japan.to_csv('Venue Details_4.csv', index=False)